In [1]:

import json

with open('networkdata.json', 'r') as fp:
    graph = json.load(fp)


In [2]:
graph['nodes'][0]

{'id': 1,
 'name': 'Hermann Otto Solms',
 'party': 'FDP',
 'nReden': 8,
 'nWorte': 761,
 'text': 'Morgen Kollegin Kollege Platz Dame Herr Kollegin Kollege Sitzung Bundestag Übung Beginn Sitzung Regelung Geschäftsordnung Bundestag Absatz Geschäftsordnung Bundestag Bundestag Mitglied Vorsitz Bundestag Präsident Fraktion Verfahren Drucksache Versammlungsleiter Sitzung Antrag Antrag Fraktion Handzeichen Gegenstimmen Enthaltung Antrag Stimme Fraktion Zustimmung AfD-Fraktion § Absatz Geschäftsordnung Herr Dr. Mitglied Bundestag Amt Alterspräsidenten Bundestag Mitglied Jahr Mitglied Bundestag Bundestag Fall Rolle Alterspräsidenten Ehre Freude Vorsitz Amtsübernahme Präsident BundestagKollege Dr.Frau Kollegin Kollege Kollege Verfahren Schatzmeister Partei Zeit Verfahren Änderung Parteiengesetzes Partei Punkt Dame Herr FDP-Fraktion Überweisung Hoffnung Koalitionsfraktionen Vernunft Diskussion Parteiengesetz Parteiengesetz Renovierung Erneuerung Stichwort Umgang Mikrospenden Überprüfung Transpare

In [95]:
def GraphVisualizer(data, properties):
    
    
    #ncol, nlabel, darkmode, edgevisibility, particles
    
    nlabel = properties['nodelabel']
    ncol = properties['nodecoloring']
    nsize = properties['nodesize']
    
    # evaluate the boolean arguments
    if properties['edgevisibility'] == True:
        lv = "//"
    else:
        lv = ""
    if properties['particles'] == False:
        parts = "//"
    else:
        parts = ""
    if properties['darkmode'] == False:
        dm = "//"
    else:
        dm = ""
    
    d3graph = {"nodes": [], "links": []}
    d3graph["nodes"] = data["nodes"]
    d3graph["links"] = data["links"]

    htmlcode = f"""<head>
        <style> body {{margin: 0;}} </style>
        <script src="https://unpkg.com/force-graph"></script>
        <meta charset="UTF-8">
    </head>
    <body>
    <div id="graph"></div>
    <script>
        var data = {d3graph};
        const elem = document.getElementById('graph');
        const Graph = ForceGraph()(elem)
            .graphData(data)
            .nodeLabel('{nlabel}')
            .nodeRelSize(12)
            .nodeAutoColorBy('{ncol}')
            {dm}.backgroundColor('#000000')
            {dm}.linkColor(() => 'rgba(255,255,255,0.2)')
            {lv}.linkVisibility('false')
            {parts}.linkDirectionalParticles(2)
            {parts}.linkDirectionalParticleWidth(1.4)
            .onNodeClick (node => {{window.open(`wordnet.html`, '_blank')}})
            //.onNodeHover(node => elem.style.cursor = node ? 'pointer' : null)
            .onNodeRightClick(node => {{
                // Center/zoom on node
                Graph.centerAt(node.x, node.y, 1000);
                Graph.zoom(8, 2000);
            }});
    </script>
    </body>
    """
    return {'graph':htmlcode}


# .onNodeClick (node => {{window.open(`https://www.universecity.de`, '_blank')}})

In [97]:

properties = {
        'nodecoloring':'maxTFIDF', 
        'nodelabel': 'name', 
        'nodesize': 'nReden',
        #'nodelabel': 'mfic',
        "darkmode": False,
        "edgevisibility": True,
        "particles": False
       }

#htmlgraph = GraphVisualizer(graph,'name','name',True,True,True)
htmlgraph = GraphVisualizer(graph,properties)

with open (f"./ParlamentarierNetzwerk.html", "w") as f:
    f.write(htmlgraph['graph'])


## Second Option using another library

In [84]:
def GraphVisualizerD3(data, properties):
    
    
    #ncol, nlabel, darkmode, edgevisibility, particles
    
    nlabel = properties['nodelabel']
    ncol = properties['nodecoloring']
    
    # evaluate the boolean arguments
    if properties['edgevisibility'] == True:
        lv = "//"
    else:
        lv = ""
    if properties['particles'] == False:
        parts = "//"
    else:
        parts = ""
    if properties['darkmode'] == False:
        dm = "//"
    else:
        dm = ""
    
    d3graph = {"nodes": [], "links": []}
    d3graph["nodes"] = data["nodes"]
    d3graph["links"] = data["links"]

    htmlcode = f"""<!DOCTYPE html>
<html>
<head>
    <script src="http://d3js.org/d3.v3.min.js" charset="utf-8"></script>
    <script src="https://d3js.org/d3.v4.js"></script>
</head>
<body>
<!-- Create a div where the graph will take place -->
<div id="upperrow"></div>
<div id="main"></div>

    <script>

// set the dimensions and margins of the graph
var margin = {{top: 10, right: 30, bottom: 30, left: 40}},
  width = 400 - margin.left - margin.right,
  height = 400 - margin.top - margin.bottom;

// append the svg object to the body of the page
var svg = d3.select("body")
.append("svg")
  .attr("width", width + margin.left + margin.right)
  .attr("height", height + margin.top + margin.bottom)
.append("g")
  .attr("transform",
        "translate(" + margin.left + "," + margin.top + ")");

d3.json("https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/data_network.json", function( data) {{

  // Initialize the links
  var link = svg
    .selectAll("line")
    .data(data.links)
    .enter()
    .append("line")
      .style("stroke", "#aaa")

  // Initialize the nodes
  var node = svg
    .selectAll("circle")
    .data(data.nodes)
    .enter()
    .append("circle")
      .attr("r", 20)
      .style("fill", "#69b3a2")

  // Let's list the force we wanna apply on the network
  var simulation = d3.forceSimulation(data.nodes)                 // Force algorithm is applied to data.nodes
      .force("link", d3.forceLink()                               // This force provides links between nodes
            .id(function(d) {{ return d.id; }})                     // This provide  the id of a node
            .links(data.links)                                    // and this the list of links
      )
      .force("charge", d3.forceManyBody().strength(-400))         // This adds repulsion between nodes. Play with the -400 for the repulsion strength
      .force("center", d3.forceCenter(width / 2, height / 2))     // This force attracts nodes to the center of the svg area
      .on("end", ticked);

  // This function is run at each iteration of the force algorithm, updating the nodes position.
  function ticked() {{
    link
        .attr("x1", function(d) {{ return d.source.x; }})
        .attr("y1", function(d) {{ return d.source.y; }})
        .attr("x2", function(d) {{ return d.target.x; }})
        .attr("y2", function(d) {{ return d.target.y; }});

    node
         .attr("cx", function (d) {{ return d.x+6; }})
         .attr("cy", function(d) {{ return d.y-6; }});
  }}

}});


</script>
</body>
</html>
        """
    return {'graph':htmlcode}






In [85]:
htmlgraph = GraphVisualizerD3(graph,properties)

with open (f"./ParlamentarierNetzwerk.html", "w") as f:
    f.write(htmlgraph['graph'])
    
    